In [49]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
import re

In [62]:
#add a year from pro football reference
def cleanFP(dfwr):
    dfwr['Player'] = dfwr['Player'].replace(to_replace="(*)", value="", regex=True, inplace=True)
    #dfwr['Fpts/G'] = dfwr.apply(lambda row: (row.Rec + row.Yds / 10 + row.TD * 6 + row.Fmb * -2) / row.G, axis=1)
    return dfwr

dfwr22 = pd.read_csv(r"C:\Users\luker\Downloads\Python\WR22.csv")
dfwr22 = cleanFP(dfwr22)
dfwr22

C:\Users\luker\AppData\Local\Temp\ipykernel_26340\2093195598.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dfwr['Player'] = dfwr['Player'].replace(to_replace="(*)", value="", regex=True, inplace=True)


error: nothing to repeat at position 1